In [4]:
from google.colab import drive
drive.mount('/content/drive')
# Google Drive einbinden
# Google Drive mounten

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import requests
import zipfile
import os
import csv
from datetime import datetime

# URL zum Herunterladen des Archivs
url = 'https://www.lotto-bayern.de/static/gamebroker_2/de/download_files/archiv_lotto.zip'

# Senden einer GET-Anfrage, um die Datei herunterzuladen
response = requests.get(url)

# Überprüfen, ob die Anfrage erfolgreich war
if response.status_code == 200:
    # Temporäre Datei zum Speichern des Archivs erstellen
    with open('/content/drive/MyDrive/Lotto/6aus49/archiv_lotto.zip', 'wb') as f:
        f.write(response.content)

    # Archiv entpacken
    with zipfile.ZipFile('/content/drive/MyDrive/Lotto/6aus49/archiv_lotto.zip', 'r') as zip_ref:
        zip_ref.extractall('/content/drive/MyDrive/Lotto/6aus49/extracted_files')

    # Inhalt der Datei .txt lesen
    txt_file = os.path.join('/content/drive/MyDrive/Lotto/6aus49/extracted_files', 'lotto.txt')
    with open(txt_file, 'r') as txtfile:
        txt_content = txtfile.readlines()

    # Inhalt in CSV konvertieren und die ersten drei Spalten ins Datumsformat "dd.mm.yyyy" umwandeln
    csv_file = os.path.join('/content/drive/MyDrive/Lotto/6aus49', '6aus49_seit2013.csv')
    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Datum', 'Zahl1', 'Zahl2', 'Zahl3', 'Zahl4', 'Zahl5', 'Zahl6', 'Superzahl'])

        # Kopfzeile der Originaldatei überspringen
        txt_content.pop(0)

        # Zeilen lesen und die ersten drei Spalten ins Datumsformat "dd.mm.yyyy" umwandeln
        for row in txt_content:
            tag, monat, jahr, *rest = row.split()  # Zeile in einzelne Werte aufteilen
            datum = datetime(int(jahr), int(monat), int(tag)).strftime('%d.%m.%Y')
            if datetime(int(jahr), int(monat), int(tag)) >= datetime(2013, 5, 4):
                writer.writerow([datum] + rest)

    print("Datei erfolgreich in CSV umgewandelt.")

    # Temporäre Dateien löschen
    os.remove('/content/drive/MyDrive/Lotto/6aus49/archiv_lotto.zip')
    os.remove(txt_file)
else:
    print("Das Archiv konnte nicht heruntergeladen werden.")


Datei erfolgreich in CSV umgewandelt.


In [6]:
import numpy as np
import pandas as pd
from datetime import datetime
import pytz
import os
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Definition der Modellparameter
model_params = {
    'batch_size': 98,
    'epochs': 666,
    'window_length': 24  # Geändert von 7 auf 24
}

# Funktion zum Laden und Vorverarbeiten der Daten
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df = df.drop(['Datum', 'Superzahl'], axis=1)
    df_sorted = df.apply(lambda row: pd.Series(sorted(row)), axis=1)
    return df_sorted

# Funktion zur Erstellung von Trainingsdaten
def create_training_data(df_sorted, window_length):
    scaler = StandardScaler().fit(df_sorted)
    transformed_df = scaler.transform(df_sorted)
    number_of_rows = df_sorted.shape[0]
    number_of_features = df_sorted.shape[1]

    X = np.empty([number_of_rows - window_length, window_length, number_of_features], dtype=float)
    y = np.empty([number_of_rows - window_length, number_of_features], dtype=float)

    for i in range(0, number_of_rows - window_length):
        X[i] = transformed_df[i : i + window_length, 0 : number_of_features]
        y[i] = transformed_df[i + window_length : i + window_length + 1, 0 : number_of_features]

    return X, y, scaler

# Funktion zur Erstellung und Kompilierung des Modells
def create_model(input_shape, learning_rate=0.001):
    model = Sequential()
    model.add(Bidirectional(LSTM(240, input_shape=input_shape, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(240, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(240, return_sequences=True)))
    model.add(Bidirectional(LSTM(240, return_sequences=False)))
    model.add(Dropout(0.2))
    model.add(Dense(49))
    model.add(Dense(input_shape[1]))  # Letzte Schicht mit Anzahl der Features als Ausgänge

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['accuracy'])
    return model

# Funktion zum Abrufen des neuesten gespeicherten Modells
def get_latest_model_path(models_dir):
    model_files = [f for f in os.listdir(models_dir) if f.endswith('.keras')]
    if not model_files:
        return None
    latest_model = max(model_files, key=lambda x: os.path.getctime(os.path.join(models_dir, x)))
    return os.path.join(models_dir, latest_model)

# Dateipfade und Parameter
data_file_path = "/content/drive/MyDrive/Lotto/6aus49/6aus49_seit2013.csv"
models_dir = "/content/drive/MyDrive/Lotto/6aus49/models"
os.makedirs(models_dir, exist_ok=True)

# Laden und Vorverarbeiten der Daten
df_sorted = load_and_preprocess_data(data_file_path)
X, y, scaler = create_training_data(df_sorted, model_params['window_length'])

# Aufteilung der Daten in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12, random_state=24)

# Versuch, das neueste Modell zu laden
model_path = get_latest_model_path(models_dir)
model = None

if model_path:
    model = load_model(model_path)
    print(f"Modell geladen: {model_path}")
    # Überprüfung der Eingabedatenform des Modells
    input_shape = (model_params['window_length'], X.shape[2])
    if model.input_shape[1:] != input_shape:
        print("Eingabedatenform geändert, neues Modell wird erstellt")
        model = create_model(input_shape)
    else:
        print("Voriges Modell mit gleicher Eingabedatenform geladen")
else:
    print("Kein Modell gefunden, neues Modell wird erstellt")
    model = create_model((model_params['window_length'], X.shape[2]))

# Aktualisierung der Modellparameter nach Erstellung/Laden des Modells
model_params.update({
    'layers': len(model.layers),
    'optimizer': str(model.optimizer),
    'loss': model.loss,
    'metrics': model.metrics_names,
})

# Training des Modells mit neuen Daten
history = model.fit(x=X_train, y=y_train, batch_size=model_params['batch_size'], epochs=model_params['epochs'], verbose=2)

# Bewertung des Modells auf Trainings- und Testdaten
train_loss, train_accuracy = model.evaluate(X_train, y_train)
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Train Loss: {train_loss}, Train Accuracy: {train_accuracy}")
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Speichern des aktualisierten Modells mit einzigartigem Namen
timezone = pytz.timezone("Europe/Berlin")
current_time = datetime.now(timezone).strftime("%Y%m%d_%H%M%S")

model_file_path = os.path.join(models_dir, f"saved_model_{current_time}.keras")
save_model(model, model_file_path)
print(f"Modell gespeichert unter: {model_file_path}")

# Beispielhafte Verwendung des Modells zur Vorhersage
to_predict = df_sorted.tail(model_params['window_length'])

# Überprüfen und Auffüllen der Daten auf die erforderliche Länge (24 Zeitschritte), falls nötig
if to_predict.shape[0] < model_params['window_length']:
    padding = np.zeros((model_params['window_length'] - to_predict.shape[0], to_predict.shape[1]))
    to_predict = np.vstack((padding, to_predict))

to_predict = np.array(to_predict)

scaled_to_predict = scaler.transform(to_predict)
y_pred = model.predict(np.array([scaled_to_predict]))
predicted_numbers = scaler.inverse_transform(y_pred).astype(int)[0]
print("Vorhergesagte Zahlen der letzten Lottoziehung:", predicted_numbers)

# Speichern der Modellparameter und vorhergesagten Zahlen in eine Datei mit einzigartigem Namen
params_file_path = os.path.join(models_dir, f"model_params_{current_time}.txt")
with open(params_file_path, 'w') as f:
    f.write(f"Train Loss: {train_loss}, Train Accuracy: {train_accuracy}\n")
    f.write(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}\n")
    f.write(f"Predicted numbers: {predicted_numbers}\n")
    f.write(f"Model parameters:\n")
    for param, value in model_params.items():
        f.write(f"{param}: {value}\n")
print(f"Modellparameter und vorhergesagte Zahlen gespeichert unter: {params_file_path}")


import requests
import json
# Replace with your actual backend URL (e.g., your Dokploy domain or Tunnel URL)
# Example: BACKEND_URL = "https://your-lotto-app.com"
BACKEND_URL = "https://apilotto.euroceiling39.ru" # Use public URL in production!
try:
    # predicted_numbers is the variable from your existing script
    payload = {
        "numbers": predicted_numbers.tolist() if hasattr(predicted_numbers, 'tolist') else list(predicted_numbers)
    }

    response = requests.post(
        f"{BACKEND_URL}/api/colab/update",
        json=payload
    )

    if response.status_code == 200:
        print("✅ Ergebnisse erfolgreich an die App gesendet!")
    else:
        print(f"❌ Fehler beim Senden: {response.status_code}")
        print(response.text)
except Exception as e:
    print(f"⚠️ Verbindungsfehler: {e}")

Modell geladen: /content/drive/MyDrive/Lotto/6aus49/models/saved_model_20260201_174836.keras
Voriges Modell mit gleicher Eingabedatenform geladen
Epoch 1/666
12/12 - 6s - 475ms/step - accuracy: 0.9609 - loss: 0.0041
Epoch 2/666
12/12 - 0s - 41ms/step - accuracy: 0.9583 - loss: 0.0044
Epoch 3/666
12/12 - 1s - 42ms/step - accuracy: 0.9530 - loss: 0.0043
Epoch 4/666
12/12 - 0s - 41ms/step - accuracy: 0.9670 - loss: 0.0042
Epoch 5/666
12/12 - 0s - 41ms/step - accuracy: 0.9522 - loss: 0.0042
Epoch 6/666
12/12 - 1s - 42ms/step - accuracy: 0.9609 - loss: 0.0040
Epoch 7/666
12/12 - 1s - 42ms/step - accuracy: 0.9583 - loss: 0.0043
Epoch 8/666
12/12 - 1s - 42ms/step - accuracy: 0.9504 - loss: 0.0043
Epoch 9/666
12/12 - 0s - 42ms/step - accuracy: 0.9496 - loss: 0.0042
Epoch 10/666
12/12 - 1s - 54ms/step - accuracy: 0.9609 - loss: 0.0042
Epoch 11/666
12/12 - 1s - 51ms/step - accuracy: 0.9600 - loss: 0.0042
Epoch 12/666
12/12 - 1s - 48ms/step - accuracy: 0.9643 - loss: 0.0043
Epoch 13/666
12/12 - 1